<a href="https://colab.research.google.com/github/mahend72/BPETIS/blob/main/BEPTIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. Homomorphic encryption**

**1.1 Install tenseal**

In [ ]:
pip install tenseal

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 15.4 MB/s eta 0:00:00


In [ ]:
def map_to_list(list1, list2):

    # Initialize a list with zeros
    IOC_severity = [0] * len(list2)

    # Map the input caste category to the list
    try:
        index = list1.index(list1)
        IOC_severity[index] = 1
    except ValueError:
        print("Invalid value.")

    return IOC_severity

**1.2 Encrypt sensitive IOCs information inclusing severity score and risk level**

In [ ]:
import tenseal as ts
import numpy as np
import time


def context():
    context = ts.context(ts.SCHEME_TYPE.CKKS, 8192, coeff_mod_bit_sizes=[60, 40, 40, 60])
    context.global_scale = pow(2, 40)
    context.generate_galois_keys()
    return context

context = context()

risk_level = ["Critical", "High", "medium", "low", "Unknown"]

input_IOC_sev = "8.055"

input_risk_level = "High"


#output_IOC_sev = map_to_list(input_IOC_sev, IOC_sev)
output_risk_level = map_to_list(input_risk_level, risk_level)


print(f"IOC severity: {input_IOC_sev}")
print(f"risk: {output_risk_level}")

start_time = time.time()

encrypted_input_IOC_sev = ts.ckks_tensor(context, [input_IOC_sev])
encrypted_input_risk_level = ts.ckks_tensor(context, ts.plain_tensor(output_risk_level, [len(output_risk_level), 1]))

print("Encrypted severity",encrypted_input_IOC_sev)
print("Encrypted risk level", encrypted_input_risk_level )

end_time = time.time()

print(end_time - start_time)

IOC severity: 8.055
risk: [1, 0, 0, 0, 0]
Encrypted severity <tenseal.tensors.ckkstensor.CKKSTensor object at 0x77fcfc4a8a30>
Encrypted risk level <tenseal.tensors.ckkstensor.CKKSTensor object at 0x77fcfc4a8a60>
0.047742366790771484


# **2. Using STIX: Serialise IOCs and structure them.**

**2.1 Install stix2**

In [ ]:
!pip install stix2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.8/177.8 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 10.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144554 sha256=945c4feaf3f661a62144c6157eef7ed9e171f3f81a7cafa07e72615946a65c17
  Stored in directory: /root/.cache/pip/wheels/12/93/dd/1f6a127edc45659556564c5730f6d4e300888f4bca2d4c5a88
Successfully built antlr4-python3-runtime


**2.2 Structuring and serialising of IOC**

In [ ]:
import stix2
import json

# Creating a simple STIX indicator with additional custom properties
risk_level = '"{}"'.format(encrypted_input_risk_level)
ioc_severity = '"{}"'.format(encrypted_input_IOC_sev)

indicator = stix2.Indicator(
    name="Malicious File Indicator",
    pattern="[file:hashes.md5 = 'd41d8cd98f00b204e9800998ecf8427e']",
    labels=["malicious-indicator"],
    pattern_type="stix",
    custom_properties={
        "risk_level": risk_level,
        "ioc_severity": ioc_severity
    }
)

# Convert STIX Indicator to JSON format
indicator_json = stix2.serialization.serialize(indicator, indent=2)

# Save JSON to a file
with open("/content/data.json", "w") as json_file:
    json_file.write(indicator_json)

print("STIX Indicator saved to 'data.json'")


STIX Indicator saved to 'data.json'


In [ ]:
ls

bin@                        datalab/  lib@     media/                    proc/        sbin@  tools/
boot/                       dev/      lib32@   mnt/                      python-apt/  srv/   usr/
content/                    etc/      lib64@   NGC-DL-CONTAINER-LICENSE  root/        sys/   var/
cuda-keyring_1.0-1_all.deb  home/     libx32@  opt/                      run/         tmp/


# **TAXII Server**

**Install Medallion**

In [ ]:
!pip install medallion

In [ ]:
ls

bin@                        datalab/  lib@     media/                    proc/        sbin@  tools/
boot/                       dev/      lib32@   mnt/                      python-apt/  srv/   usr/
content/                    etc/      lib64@   NGC-DL-CONTAINER-LICENSE  root/        sys/   var/
cuda-keyring_1.0-1_all.deb  home/     libx32@  opt/                      run/         tmp/


**Run medallion**

In [ ]:
!python /usr/local/lib/python3.10/dist-packages/medallion/scripts/run.py /content/medallion-config.json

 * Serving Flask app 'medallion'
 * Debug mode: off
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
^C


# **TAXII client2**

**Install taxii2-client**

In [ ]:
!pip install taxii2-client

In [ ]:
from taxii2client import Server

# Connect to MITRE ATT&CK TAXII server
taxii_server_url = 'https://localhost/'
taxii_server = Server(taxii_server_url)

# Discover services
api_roots = taxii_server.api_roots
for api_root in api_roots:
    print("API Root:", api_root.title)
    print("Available Services:", api_root.versions)

localhost found


ConnectionError: HTTPSConnectionPool(host='localhost', port=443): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7eb648555a50>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [6]:
import json
from taxii2client import Server, Collection

def read_cti_from_file(file_path):
    with open(file_path, 'r') as file:
        cti_content = json.load(file)
    return cti_content

def publish_cti_from_file(taxii_server_url, collection_id, username, password, file_path):
    # Connect to the TAXII server
    server = Server(taxii_server_url, user=username, password=password)

    # Choose the collection to publish to
    collection = server.get_collection(collection_id)

    # Read CTI content from the JSON file
    cti_content = read_cti_from_file(file_path)

    # Publish the CTI content to the collection
    status = collection.add_objects(cti_content)

    print("CTI published successfully. Status:", status)

if __name__ == "__main__":
    # Replace with your TAXII server details
    taxii_server_url = "localhost"

    # Replace with your collection ID
    collection_id = "your_collection_id"

    # Replace with your TAXII server credentials
    username = "guest"
    password = "guest"

    # Specify the path to your JSON file
    file_path = "data.json"

    publish_cti_from_file(taxii_server_url, collection_id, username, password, file_path)


ModuleNotFoundError: No module named 'taxii2client'

In [ ]:
from taxii2client import Server, Collection

def consume_cti(taxii_server_url, collection_id, username, password):
    # Connect to the TAXII server
    server = Server(taxii_server_url, user=username, password=password)

    # Choose the collection to consume from
    collection = server.get_collection(collection_id)

    # Retrieve STIX objects from the collection
    stix_bundle = collection.get_objects()

    # Print the STIX objects
    print("STIX Objects:")
    for stix_object in stix_bundle['objects']:
        print(json.dumps(stix_object, indent=2))
        print("\n" + "=" * 50 + "\n")  # Separator between objects

if __name__ == "__main__":
    # Replace with your TAXII server details
    taxii_server_url = "localhost"

    # Replace with your collection ID
    collection_id = "your_collection_id"

    # Replace with your TAXII server credentials
    username = "guest"
    password = "guest"

    consume_cti(taxii_server_url, collection_id, username, password)


# **5. Check IOC severity**

In [ ]:
import tenseal as ts

# Create a CKKS context
context = ts.context(ts.SCHEME_TYPE.CKKS, 8192, coeff_mod_bit_sizes=[60, 40, 40, 60])
context.global_scale = pow(2, 40)
context.generate_galois_keys()

# Encrypt the numbers

number2 = 3.0
number3 = 6.0
encrypted_number1 = encrypted_input_IOC_sev
encrypted_number2 = ts.ckks_tensor(context, [number2])
encrypted_number3 = ts.ckks_tensor(context, [number3])

print(encrypted_input_IOC_sev.decrypt())
# Subtract the numbers (effectively performing a comparison)
result_tensor12 = encrypted_number2 - encrypted_number1
result_tensor13 = encrypted_number3 - encrypted_number1
print(result_tensor12)

# Decrypt the difference
decrypted_result12 = result_tensor12.decrypt().tolist()[0]  # Correct extraction
decrypted_result13 = result_tensor13.decrypt().tolist()[0]  # Correct extraction
print(decrypted_result12)

# Interpret the decrypted result to determine which number is greater

if decrypted_result13 < 0:
  print("Critical IOC")
elif decrypted_result12 < 0 and decrypted_result13 > 0:
  print("High priority IOC")
else: print("Low priority IOC")


3.1800891609704903e+31
Low priority IOC


 **Install pikka**

In [ ]:
!python -m pip install pika --upgrade

**Show pika information**

In [ ]:
pip show pika

Name: pika
Version: 1.3.2
Summary: Pika Python AMQP Client Library
Home-page: 
Author: 
Author-email: 
License: BSD-3-Clause
Location: /usr/local/lib/python3.10/dist-packages
Requires: 
Required-by: 


**Install rabbitmq server**

In [ ]:
!sudo apt-get install rabbitmq-server

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  erlang-asn1 erlang-base erlang-crypto erlang-eldap erlang-ftp erlang-inets
  erlang-mnesia erlang-os-mon erlang-parsetools erlang-public-key
  erlang-runtime-tools erlang-snmp erlang-ssl erlang-syntax-tools erlang-tftp
  erlang-tools erlang-xmerl libsctp1 libwrap0 logrotate socat
Suggested packages:
  erlang erlang-manpages erlang-doc lksctp-tools bsd-mailx | mailx
The following NEW packages will be installed:
  erlang-asn1 erlang-base erlang-crypto erlang-eldap erlang-ftp erlang-inets
  erlang-mnesia erlang-os-mon erlang-parsetools erlang-public-key
  erlang-runtime-tools erlang-snmp erlang-ssl erlang-syntax-tools erlang-tftp
  erlang-tools erlang-xmerl libsctp1 libwrap0 logrotate rabbitmq-server socat
0 upgraded, 22 newly installed, 0 to remove and 24 not upgraded.
Need to get 35.4 MB of archives.
After this operation, 56.3 MB of addi

**Start rabbitmq server**

In [ ]:
!sudo service rabbitmq-server restart

 * Restarting RabbitMQ Messaging Server rabbitmq-server
   ...done.


**check plugins list (if required)**

In [ ]:
!rabbitmq-plugins list

Listing plugins with pattern ".*" ...
 Configured: E = explicitly enabled; e = implicitly enabled
 | Status: * = running on rabbit@905cfacc9708
 |/
[  ] rabbitmq_amqp1_0                  3.9.13
[  ] rabbitmq_auth_backend_cache       3.9.13
[  ] rabbitmq_auth_backend_http        3.9.13
[  ] rabbitmq_auth_backend_ldap        3.9.13
[  ] rabbitmq_auth_backend_oauth2      3.9.13
[  ] rabbitmq_auth_mechanism_ssl       3.9.13
[  ] rabbitmq_consistent_hash_exchange 3.9.13
[  ] rabbitmq_event_exchange           3.9.13
[  ] rabbitmq_federation               3.9.13
[  ] rabbitmq_federation_management    3.9.13
[  ] rabbitmq_jms_topic_exchange       3.9.13
[  ] rabbitmq_management               3.9.13
[  ] rabbitmq_management_agent         3.9.13
[  ] rabbitmq_mqtt                     3.9.13
[  ] rabbitmq_peer_discovery_aws       3.9.13
[  ] rabbitmq_peer_discovery_common    3.9.13
[  ] rabbitmq_peer_discovery_consul    3.9.13
[  ] rabbitmq_peer_discovery_etcd      3.9.13
[  ] rabbitmq_peer_disco

**Enable  rabbitmq_management (if disabled)**

In [ ]:
!rabbitmq-plugins enable rabbitmq_management

Enabling plugins on node rabbit@905cfacc9708:
rabbitmq_management
The following plugins have been configured:
  rabbitmq_management
  rabbitmq_management_agent
  rabbitmq_web_dispatch
Applying plugin configuration to rabbit@905cfacc9708...
The following plugins have been enabled:
  rabbitmq_management
  rabbitmq_management_agent
  rabbitmq_web_dispatch

started 3 plugins.


**On sender side**

In [ ]:
import pika
import time

# Function to read the content of a file
def read_file_content(file_path):
    with open(file_path, 'r') as file:
        content = file.read()
    return content

# Specify the path to the file you want to read
file_path = 'data.json'

# Record the start time
start_time = time.time()

# Create a connection to RabbitMQ server
connection = pika.BlockingConnection(pika.ConnectionParameters('localhost'))
channel = connection.channel()

# Declare a queue
channel.queue_declare(queue='queue1')

# Read the content of the file
file_content = read_file_content(file_path)

# Publish the file content as a message to the queue
channel.basic_publish(exchange='', routing_key='queue1', body=file_content)

# Record the end time
end_time = time.time()

# Calculate the computation time
computation_time = end_time - start_time
#print(body)
print(f"Message published from file content in {computation_time:.6f} seconds")


# Close the connection
connection.close()


Message published from file content in 0.020598 seconds


**On reciever side**

In [ ]:
import pika

# Record the start time
start_time = time.time()


# Create a connection to RabbitMQ server
connection = pika.BlockingConnection(pika.ConnectionParameters('localhost'))
channel = connection.channel()

# Declare the same queue
channel.queue_declare(queue='queue1')

# Define a callback function for handling received messages
def callback(ch, method, properties, body):
    print(f"Received message: {body}")
    print(f"Message received in {computation_time:.6f} seconds")

# Set up the consumer to use the callback function
channel.basic_consume(queue='queue1', on_message_callback=callback, auto_ack=True)

# Record the end time
end_time = time.time()


# Calculate the computation time
computation_time = end_time - start_time
#print(f"Message received in {computation_time:.6f} seconds")

# Start consuming messages
print('Waiting for messages. To exit, press CTRL+C')
channel.start_consuming()




Waiting for messages. To exit, press CTRL+C
Received message: b'{\n  "type": "indicator",\n  "spec_version": "2.1",\n  "id": "indicator--4cddc150-e2c4-4695-9537-ad1ec7c992a6",\n  "created": "2024-01-05T07:45:46.564075Z",\n  "modified": "2024-01-05T07:45:46.564075Z",\n  "name": "Malicious File Indicator",\n  "pattern": "[file:hashes.md5 = \'d41d8cd98f00b204e9800998ecf8427e\']",\n  "pattern_type": "stix",\n  "pattern_version": "2.1",\n  "valid_from": "2024-01-05T07:45:46.564075Z",\n  "labels": [\n    "malicious-indicator"\n  ],\n  "ioc_severity": "\\"<tenseal.tensors.ckkstensor.CKKSTensor object at 0x7eb6484d9480>\\"",\n  "risk_level": "\\"<tenseal.tensors.ckkstensor.CKKSTensor object at 0x7eb648498e80>\\""\n}'
Message received in 0.015556 seconds


KeyboardInterrupt: ignored

In [ ]:
import pika
import time

# Function to read the content of a file
def read_file_content(file_path):
    with open(file_path, 'r') as file:
        content = file.read()
    return content

# Specify the path to the file you want to read
file_path = '/content/data.json'

# Number of messages to send
num_messages = 10

# Target send rate (messages per second)
target_send_rate = 5

# Calculate the delay between messages to achieve the target send rate
delay_between_messages = 1 / target_send_rate

# Create a connection to RabbitMQ server
connection = pika.BlockingConnection(pika.ConnectionParameters('localhost'))
channel = connection.channel()

# Declare a queue
channel.queue_declare(queue='queue1')

# Loop to send messages
for i in range(num_messages):
    # Read the content of the file
    file_content = read_file_content(file_path)

    # Publish the file content as a message to the queue
    channel.basic_publish(exchange='', routing_key='queue1', body=file_content)

    print(f"Message {i+1} published from file content")

    # Introduce a delay between messages to achieve the target send rate
    time.sleep(delay_between_messages)

# Close the connection
connection.close()


Message 1 published from file content
Message 2 published from file content
Message 3 published from file content
Message 4 published from file content
Message 5 published from file content
Message 6 published from file content
Message 7 published from file content
Message 8 published from file content
Message 9 published from file content
Message 10 published from file content


In [ ]:
import pika
import time

# Function to read the content of a file
def read_file_content(file_path):
    with open(file_path, 'r') as file:
        content = file.read()
    return content

# Specify the path to the file you want to read
file_path = '/content/data.json'

# Number of messages to send
num_messages = 100

# Target send rate (messages per second)
target_send_rate = 75

# Calculate the delay between messages to achieve the target send rate
delay_between_messages = 1 / target_send_rate

# Create a connection to RabbitMQ server
connection = pika.BlockingConnection(pika.ConnectionParameters('localhost'))
channel = connection.channel()

# Declare a queue
channel.queue_declare(queue='queue1')

# Record the start time
start_time = time.time()

# Loop to send messages
for i in range(num_messages):
    # Read the content of the file
    file_content = read_file_content(file_path)

    # Publish the file content as a message to the queue
    channel.basic_publish(exchange='', routing_key='queue1', body=file_content)

    print(f"Message {i+1} published from file content")

    # Introduce a delay between messages to achieve the target send rate
    time.sleep(delay_between_messages)

# Record the end time
end_time = time.time()

# Calculate the actual send rate
actual_send_rate = num_messages / (end_time - start_time)

print(f"Target Send Rate: {target_send_rate} messages per second")
print(f"Actual Send Rate: {actual_send_rate:.2f} messages per second")

# Close the connection
connection.close()


Message 1 published from file content
Message 2 published from file content
Message 3 published from file content
Message 4 published from file content
Message 5 published from file content
Message 6 published from file content
Message 7 published from file content
Message 8 published from file content
Message 9 published from file content
Message 10 published from file content
Message 11 published from file content
Message 12 published from file content
Message 13 published from file content
Message 14 published from file content
Message 15 published from file content
Message 16 published from file content
Message 17 published from file content
Message 18 published from file content
Message 19 published from file content
Message 20 published from file content
Message 21 published from file content
Message 22 published from file content
Message 23 published from file content
Message 24 published from file content
Message 25 published from file content
Message 26 published from file con